In [1]:
#필수 모듈 불러오기
from keras.models import load_model
import cv2
import numpy as np
import serial

#시리얼 통신 객체 생성
ser = serial.Serial('COM29', 9600)

#데이터 전처리
np.set_printoptions(suppress=True)

#모델 불러오기
model = load_model("keras_Model.h5", compile=False)

#라벨 불러오기
class_names = open("labels.txt", "r").readlines()

#카메라 객체 생성
camera = cv2.VideoCapture(0)

#width*height 해상도로 변경하기
camera.set(cv2.CAP_PROP_FRAME_WIDTH, 300)
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 300)

while True:
    # 카메라읽기
    ret, image = camera.read()
    
    #좌우 반전
    image=cv2.flip(image,1)
    
    #데이터 전처리
    data = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
    data = np.asarray(data, dtype=np.float32).reshape(1, 224, 224, 3)
    data = (data / 127.5) - 1

    #모델로 예측,판단결과지정
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]
    
    #확률이 일정수치 이상일때 실행
    if confidence_score >= 0.7:
        #라벨,점수 형식맞추기
        label = class_name[2 : -1]
        score = np.round(confidence_score * 100)
        text = f"{label}: {score}%"
        
        #창에띄우기
        cv2.putText(image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        #시리얼통신(보내기)
        ser.write(label[0].encode())
    
    #창띄우기
    cv2.imshow("Webcam Image", image)
    
    #q누르면빠져나가기
    if cv2.waitKey(1) == ord('q'):
        break

#사용한 카메라, 창, 시리얼통신 닫기
ser.close()
camera.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 22ms/step
